In [1]:
!pip install --upgrade pip --user
!pip install jupyter-dash dash dash_leaflet plotly pandas --user

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [2]:
# main file

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the modified AnimalShelter class
from CRUDquery import AnimalShelter  # Adjust the import statement according to your file structure

# Data Manipulation / Model
db = AnimalShelter('aac_shelter_outcomes.csv')  # Use the CSV file
df = pd.DataFrame.from_records(db.read({}))

# Print column names for verification
print(df.columns)

# Ensure correct column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Print column names after standardization
print(df.columns)

# Dashboard Layout / View
app = JupyterDash(__name__)

# Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'  # Replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.B(html.H1('SNHU CS-499 Artifact Three Dashboard - Camila Lopez'))),
    html.Hr(),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Div([
        html.Label('Filter by Rescue Type'),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster Rescue or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            labelStyle={'display': 'inline-block'}
        ),
    ]),
    html.Div([
        html.Label('Filter by Animal Type'),
        dcc.Dropdown(
            id='animal-type',
            options=[{'label': i, 'value': i} for i in df['animal_type'].unique()],
            multi=True,
        ),
    ]),
    html.Div([
        html.Label('Filter by Breed'),
        dcc.Dropdown(
            id='breed',
            options=[{'label': i, 'value': i} for i in df['breed'].unique()],
            multi=True,
        ),
    ]),
    html.Div([
        html.Label('Search by Name'),
        dcc.Input(id='name-search', type='text', placeholder='Enter name')
    ]),
    html.Div([
        html.Label('Filter by Age (weeks)'),
        dcc.Input(id='min-age', type='number', placeholder='Min age (weeks)', min=0),
        dcc.Input(id='max-age', type='number', placeholder='Max age (weeks)', min=0)
    ]),
    html.Button('Update', id='update-button', n_clicks=0),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         row_selectable="single",
                         selected_rows=[0],
                         editable=True,
                         filter_action="native",
                         sort_action="native",
                         page_size=10,
                         page_action="native",
                         row_deletable=True,
                         ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

# Interaction Between Components / Controller

@app.callback(Output('datatable-id', 'data'),
              [Input('update-button', 'n_clicks')],
              [State('filter-type', 'value'),
               State('animal-type', 'value'),
               State('breed', 'value'),
               State('name-search', 'value'),
               State('min-age', 'value'),
               State('max-age', 'value')])
def update_dashboard(n_clicks, filter_type, animal_type, breed, name_search, min_age, max_age):
    query = {}
    try:
        if filter_type and filter_type != 'Reset':
            if filter_type == 'Water Rescue':
                query.update(db.water_rescue_query())
            elif filter_type == 'Mountain or Wilderness Rescue':
                query.update(db.mountain_rescue_query())
            elif filter_type == 'Disaster Rescue or Individual Tracking':
                query.update(db.disaster_rescue_query())

        if animal_type:
            query['animal_type'] = {'$in': animal_type}
        if breed:
            query['breed'] = {'$in': breed}
        if name_search:
            query['name'] = name_search
        if min_age is not None:
            query['age_upon_outcome_in_weeks'] = {'$gte': min_age}
        if max_age is not None:
            if 'age_upon_outcome_in_weeks' in query:
                query['age_upon_outcome_in_weeks']['$lte'] = max_age
            else:
                query['age_upon_outcome_in_weeks'] = {'$lte': max_age}

        data = db.read(query) if query else df.to_dict('records')
        if data is None:
            data = df.to_dict('records')
    except Exception as e:
        print(f"Error: {str(e)}")
        data = df.to_dict('records')

    return data

# Display the breeds of animal based on quantity represented in the data table
@app.callback(Output('graph-id', "children"),
              [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if not viewData:
        return []
    df_view = pd.DataFrame(viewData)
    return [
        dcc.Graph(
            figure=px.pie(df_view, names='breed', title='Preferred Animals')
        )
    ]

# Highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if not viewData or not index:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty or not index:
        return []
    
    row = index[0]
    
    # Use actual column names from your dataset
    latitude_col = 'location_lat'  # Adjust this according to the actual column name
    longitude_col = 'location_long'  # Adjust this according to the actual column name
    name_col = 'name'  # Adjust this according to the actual column name
    breed_col = 'breed'  # Adjust this according to the actual column name

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row][latitude_col], dff.iloc[row][longitude_col]], children=[
                dl.Tooltip(dff.iloc[row][breed_col]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row][name_col])
                ])
            ])
        ])
    ]

if __name__ == '__main__':
    app.run_server(debug=True)


Index(['Unnamed: 0', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed',
       'color', 'date_of_birth', 'datetime', 'monthyear', 'name',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat',
       'location_long', 'age_upon_outcome_in_weeks'],
      dtype='object')
Index(['unnamed:_0', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed',
       'color', 'date_of_birth', 'datetime', 'monthyear', 'name',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat',
       'location_long', 'age_upon_outcome_in_weeks'],
      dtype='object')


\\apporto.com\dfs\SNHU\Users\camilalopez_snhu\AppData\Roaming\Python\Python37\site-packages\dash\dash.py:539: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Dash app running on http://127.0.0.1:8050/
Error:  'Outcome Type'
